# Delinquent Loan: how to find them?

![Image of Yaktocat](https://www.usnews.com/dims4/USNEWS/a10d42c/2147483647/thumbnail/970x647/quality/85/?url=http%3A%2F%2Fcom-usnews-beam-media.s3.amazonaws.com%2F79%2Ffa%2F1ca61f7246838f7442e1da1613a9%2F151203pastduebill-stock.jpg)

As described in [fundera.com](https://www.fundera.com/blog/delinquent-loan), a *loan becomes delinquent as soon as you’re late on a payment, by even one day. If you miss several payments or can’t make payments for an extended time (usually 90 to 120 days), the lender will place the loan in default and can start collection proceedings against you. Both delinquencies and defaults damage your credit*, which is why it would be extremely convenient to predict, according to specific loan's characteristics, whether the loan will become delinquer. To this end one can think to build a machine leraning model that can accurately predict if a borrower will pay off their loan on time or not.

In order to build such a machine learning model we will consider the Lending Club releases data for all of the approved and declined loan applications periodically on their [website](https://www.lendingclub.com/info/download-data.action). Together with these data there is also a **data dictionary** hosted on [Google Drive](https://docs.google.com/spreadsheets/d/191B2yJ4H1ZPXq0_ByhUgWMFZOYem5jFz0Y3by_7YBY4/edit#gid=2081333097). Lending Club is a marketplace for personal loans that matches borrowers who are seeking a loan with investors looking to lend money and make a return. Each borrower fills out a comprehensive application, providing their past financial history, the reason for the loan, and more.

The LoanStats sheet describes the approved loans datasets and the RejectStats describes the rejected loans datasets. Since rejected applications don't appear on the Lending Club marketplace and aren't available for investment, we'll be focusing on data on approved loans only. In particular we will consider loan data from 2007 to 2011, since a good number of the loans have already finished.

The approved loans datasets contain information on current loans, completed loans, and defaulted loans. 

### Summary of the Conclusions

After a proper cleaning of the dataset, the problem under analysis was modeled as a binary classification and solved by using logistic regression. Without assigning any harsher penalty for misclassifying a specific class (because of the unbalancing classes problem), we were able to obtain a high precision without loosing a lot of 'good' borrowers.

## Preliminary Steps

In [1]:
# useful imports
import pandas as pd
import numpy as np
pd.set_option('max_columns', 120)
pd.set_option('max_colwidth', 5000)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,8)
from IPython.display import HTML

# Machine Learning
from sklearn.metrics import precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

Let us define useful functions for dataframe visualization

In [2]:
def hover(hover_color="#ffff99"):
    return dict(selector="tr:hover",
                props=[("background-color", "%s" % hover_color)])

In [3]:
def df2table(df):
    styles = [
        hover(),
        dict(selector="th", props=[("font-size", "150%"),
                                   ("text-align", "center")]),
        dict(selector="caption", props=[("caption-side", "bottom")])
    ]
    html = (df.style.set_table_styles(styles)
              .set_caption("Hover to highlight."))
    return html

## Data Cleaning

Once the dataset has been imported and analyzed, one of the most important step in order to build a predictive model is data cleaning and the selection of the most useful features to address our project goal.

In [4]:
#import dataset
loans_2007 = pd.read_csv("loans_2007.csv")
loans_description = pd.read_excel ("LCDataDictionary.xlsx")

/Users/jessicalanini/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
#Briefly evaluation of the dataset
df2table(loans_2007.head())

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1.2966e+06,5000,5000,4975,36 months,10.65%,162.87,B,B2,nan,10+ years,RENT,24000,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0,Jan-1985,1,3,0,13648,83.7%,9,f,0,0,5863.16,5833.84,5000,863.16,0,0,0,Jan-2015,171.62,Jun-2016,0,1,INDIVIDUAL,0,0,0,0,0
1,1077430,1.31417e+06,2500,2500,2500,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1,0,Apr-1999,5,3,0,1687,9.4%,4,f,0,0,1008.71,1008.71,456.46,435.17,0,117.08,1.11,Apr-2013,119.66,Sep-2013,0,1,INDIVIDUAL,0,0,0,0,0
2,1077175,1.31352e+06,2400,2400,2400,36 months,15.96%,84.33,C,C5,nan,10+ years,RENT,12252,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0,Nov-2001,2,2,0,2956,98.5%,10,f,0,0,3005.67,3005.67,2400,605.67,0,0,0,Jun-2014,649.91,Jun-2016,0,1,INDIVIDUAL,0,0,0,0,0
3,1076863,1.27718e+06,10000,10000,10000,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20,0,Feb-1996,1,10,0,5598,21%,37,f,0,0,12231.9,12231.9,10000,2214.92,16.97,0,0,Jan-2015,357.48,Apr-2016,0,1,INDIVIDUAL,0,0,0,0,0
4,1075358,1.31175e+06,3000,3000,3000,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0,Jan-1996,0,15,0,27783,53.9%,38,f,461.73,461.73,3581.12,3581.12,2538.27,1042.85,0,0,0,Jun-2016,67.79,Jun-2016,0,1,INDIVIDUAL,0,0,0,0,0


In [6]:
loans_2007.shape

(42538, 52)

The Dataframe contains many columns (i.e   . features) and we need to understand which of them:

- leak information from the future (introduction of information about the target, which should not be legitimately available to mine from, e.g. informations related to a period after the loan has already been funded)
- don't affect a borrower's ability to pay back a loan (e.g. a randomly generated ID value by Lending Club)
- formatted poorly and need to be cleaned up
- require more data or a lot of processing to turn into a useful feature
- contain redundant information

Let's break up the columns into 3 groups of 18 columns and use the data dictionary to become familiar with what each column represents. 

In [7]:
# create table with the name of the features in the original dataframe, their corresponding type, 
#first value and description according to the LCDataDictionary.xlsx file 
cols = loans_2007.columns.values
table_summary = pd.DataFrame()
table_summary['name'] = cols
table_summary['dtype'] = loans_2007[cols].dtypes.values
table_summary['first value'] = loans_2007[cols].iloc[0].values
table_summary = table_summary.merge(loans_description, left_on='name', right_on='LoanStatNew')
table_summary.drop("LoanStatNew", axis = 1, inplace = True)
df2table(table_summary[:18])

,name,dtype,first value,Description
0,id,object,1077501,A unique LC assigned ID for the loan listing.
1,member_id,float64,1.2966e+06,A unique LC assigned Id for the borrower member.
2,loan_amnt,float64,5000,"The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value."
3,funded_amnt,float64,5000,The total amount committed to that loan at that point in time.
4,funded_amnt_inv,float64,4975,The total amount committed by investors for that loan at that point in time.
5,term,object,36 months,The number of payments on the loan. Values are in months and can be either 36 or 60.
6,int_rate,object,10.65%,Interest Rate on the loan
7,installment,float64,162.87,The monthly payment owed by the borrower if the loan originates.
8,grade,object,B,LC assigned loan grade
9,sub_grade,object,B2,LC assigned loan subgrade


In [8]:
df2table(table_summary[18:37])  

,name,dtype,first value,Description
18,purpose,object,credit_card,A category provided by the borrower for the loan request.
19,title,object,Computer,The loan title provided by the borrower
20,zip_code,object,860xx,The first 3 numbers of the zip code provided by the borrower in the loan application.
21,addr_state,object,AZ,The state provided by the borrower in the loan application
22,dti,float64,27.65,"A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income."
23,delinq_2yrs,float64,0,The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years
24,earliest_cr_line,object,Jan-1985,The month the borrower's earliest reported credit line was opened
25,inq_last_6mths,float64,1,The number of inquiries in past 6 months (excluding auto and mortgage inquiries)
26,open_acc,float64,3,The number of open credit lines in the borrower's credit file.
27,pub_rec,float64,0,Number of derogatory public records


In [9]:
df2table(table_summary[37:])

,name,dtype,first value,Description
37,total_rec_int,float64,863.16,Interest received to date
38,total_rec_late_fee,float64,0,Late fees received to date
39,recoveries,float64,0,post charge off gross recovery
40,collection_recovery_fee,float64,0,post charge off collection fee
41,last_pymnt_d,object,Jan-2015,Last month payment was received
42,last_pymnt_amnt,float64,171.62,Last total payment amount received
43,last_credit_pull_d,object,Jun-2016,The most recent month LC pulled credit for this loan
44,collections_12_mths_ex_med,float64,0,Number of collections in 12 months excluding medical collections
45,policy_code,float64,1,publicly available policy_code=1 new products not publicly available policy_code=2
46,application_type,object,INDIVIDUAL,Indicates whether the loan is an individual application or a joint application with two co-borrowers


After analysing all the features in the previous table we can select the ones that should be removed from the original dataframe and those that may be just modified for the next steps. 
In particular, the features that should be removed are the following:
- *id* and *member_id* because they don't affect a borrower's ability to pay back a loan
- *emp_title* may be relevant but in general we should convert it into a categorical variable and we should also consider that there are not only a predefine set of job title, but that eventually, a new borrower may have another job that was not defined in the specific set we have for building the model;
- *issue_d* leaks data from the future (after the loan is already completed funded);
- *title*, for the same reason of *emp_title*;
- *zip_code* and *addr_state*, in general, may be relevant but I believe only if the loan system is restricted to a specific area. If not, a new possible borrower may come from another place that was not in the initial analyzed set and the algorithm would not be able to apportunely process this information.
- *earliest_cr_line* has no information for our purpose;
- *total_rec_late_fee*, *total_rec_int*, *out_prncp*, *out_prncp_inv, total_pymnt, total_pymnt_inv, recoveries, last_pymnt_amnt, collection_recovery_fee, delinq_amnt, total_rec_prncp, funded_amnt and funded_amnt_inv* give leak information from the future; 
- *last_pymnt_d, last_credit_pull_d* does not give useful information;
- *grade* and *sub_grade* contains redundant information as the interest rate column (*int_rate*)



The features that should be converted are:
- *term*, where we can remove the 'month' word and consider only the number for post evaluations;
- *int_rate*, where we need to remove the '%' symbol;
- *home_ownership* should be converted into categorical;
- *pymnt_plan* should be categorical;
- *revol_util*, where the '%' symbol can be removed;
- *initial_list_status*, needs to be converted into categorical;
- *policy_code* and *application_type*, should be converted into categorical;
- *purpose*, convert to categorical;


#### Remove Features

In [10]:
# Insert in a list all the columns that should be dropped  
cols = ['id', 'member_id', 'funded_amnt', 'funded_amnt_inv', 'grade', 'sub_grade', 'emp_title', 
        'issue_d', 'zip_code', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 
        'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 
        'collection_recovery_fee', 'last_pymnt_amnt', 'last_pymnt_d']
loans_2007.drop(cols, axis = 1, inplace = True)
extra_cols =['title', 'addr_state', 'earliest_cr_line', 'delinq_amnt', 'last_credit_pull_d'] 
loans_2007.drop(extra_cols, axis = 1, inplace = True)

In [11]:
loans_2007.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,pub_rec_bankruptcies,tax_liens
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,Fully Paid,n,credit_card,27.65,0.0,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.0,Source Verified,Charged Off,n,car,1.00,0.0,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.0,Not Verified,Fully Paid,n,small_business,8.72,0.0,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0
3,10000.0,36 months,13.49%,339.31,10+ years,RENT,49200.0,Source Verified,Fully Paid,n,other,20.00,0.0,1.0,10.0,0.0,5598.0,21%,37.0,f,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0
4,3000.0,60 months,12.69%,67.79,1 year,RENT,80000.0,Source Verified,Current,n,other,17.94,0.0,0.0,15.0,0.0,27783.0,53.9%,38.0,f,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0


In [12]:
loans_2007.shape

(42538, 27)

After removing the selected columns we can evaluate other columns to be removed by considering those ones that contain only one unique value, since they are not informative in this study. To this end let us define a function **drop_unique**.

In [13]:
def drop_unique(df, col_name):
    # first remove NaN values
    non_null = df[col_name].dropna()
    # second let us count the unique values
    unique_non_null = non_null.unique()
    num_true_unique = len(unique_non_null)
    col_dropped = 'none'
    if (num_true_unique ==1):
        df.drop(col_name, axis =1, inplace = True)
        col_dropped = col_name
    return df, col_dropped

In [14]:
# Let us define an empty array to collect the columns name to be dropped
collect_drop_col = []
# For all the columns of the dataframe we will drop the columns
#where the value of unique values is =1
cols = loans_2007.columns
for col in cols:
    loans_2007, col_dropped = drop_unique(loans_2007, col)
    if (col_dropped != 'none'):
        collect_drop_col.append(col_dropped)

In [15]:
print 'The columns with a single unique value are:', collect_drop_col

The columns with a single unique value are: ['initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'chargeoff_within_12_mths']


In [16]:
loans_2007.shape

(42538, 22)

We were able to remove other 5 columns from the original dataset.

#### Missing Values

In order to use the data store in **loans_2007** dataframe for the development of a predictive mode, it is important to remove all the null values. Indeed the mathematics underlying most machine learning models assumes that the data is numerical and contains no missing values.

In [17]:
# Return the number of missing values in the dataframe
missing_values = loans_2007.isnull().sum()
missing_values

loan_amnt                  3
term                       3
int_rate                   3
installment                3
emp_length              1115
home_ownership             3
annual_inc                 7
verification_status        3
loan_status                3
pymnt_plan                 3
purpose                    3
dti                        3
delinq_2yrs               32
inq_last_6mths            32
open_acc                  32
pub_rec                   32
revol_bal                  3
revol_util                93
total_acc                 32
acc_now_delinq            32
pub_rec_bankruptcies    1368
tax_liens                108
dtype: int64

As we can see there are high amount of missing data for two features: *emp_length* and *pub_rec_bankruptcies*. Employment length is frequently used in assessing how risky a potential borrower is, so we'll keep this column despite its relatively large amount of missing values. 

Let's ispect the *pub_rec_bankruptcies* feature.

In [18]:
loans_2007['pub_rec_bankruptcies'].value_counts(normalize = True, dropna = False)

 0.0    0.924256
 1.0    0.043396
NaN     0.032159
 2.0    0.000188
Name: pub_rec_bankruptcies, dtype: float64

As one can see above, since almost all the values (92%) belong to the same category, this feature would not have the right variability to help us in the construction of the predictive model. For this reason we can drop it.

In [19]:
loans_2007.drop('pub_rec_bankruptcies', axis = 1, inplace = True)

For the other columns we can keep them and just remove the rows where they have null values.

In [20]:
# drop_na from the dataframe
loans_2007.dropna(inplace = True)
loans_2007.shape

(41261, 21)

In [21]:
# Check if no NaN value is in the dataframe
loans_2007.isnull().sum()

loan_amnt              0
term                   0
int_rate               0
installment            0
emp_length             0
home_ownership         0
annual_inc             0
verification_status    0
loan_status            0
pymnt_plan             0
purpose                0
dti                    0
delinq_2yrs            0
inq_last_6mths         0
open_acc               0
pub_rec                0
revol_bal              0
revol_util             0
total_acc              0
acc_now_delinq         0
tax_liens              0
dtype: int64

#### Convert Object Features

Once all the null values have been removed from the dataframe, we need to manipulate the non-numeric values. 
Although we have previously individauted the features which need to be converted, we can precisely detect them as follows:

In [22]:
# Select variables of 'object' type
object_columns_df = loans_2007.select_dtypes(include=['object'])
df2table(object_columns_df.head())

,term,int_rate,emp_length,home_ownership,verification_status,loan_status,pymnt_plan,purpose,revol_util
0,36 months,10.65%,10+ years,RENT,Verified,Fully Paid,n,credit_card,83.7%
1,60 months,15.27%,< 1 year,RENT,Source Verified,Charged Off,n,car,9.4%
2,36 months,15.96%,10+ years,RENT,Not Verified,Fully Paid,n,small_business,98.5%
3,36 months,13.49%,10+ years,RENT,Source Verified,Fully Paid,n,other,21%
4,60 months,12.69%,1 year,RENT,Source Verified,Current,n,other,53.9%


As we can see some of these features can be quickly converted into numerical values. In particular columns *int_rate* and *revol_util* where a percentage is indicated. 

In [23]:
# remove percentage symbol from features int_rate and revol_util
cols = ['int_rate', 'revol_util']
for col in cols:
    loans_2007[col] = loans_2007[col].astype(str).apply(lambda x: x.replace('%', '')).astype('float')
loans_2007[cols].head()

,int_rate,revol_util
0,10.65,83.7
1,15.27,9.4
2,15.96,98.5
3,13.49,21.0
4,12.69,53.9


Let's now explore the unique values counts of the columns in order to check which feature can be considered categorical, in particular *emp_length*, *home_ownership*, *verification_status*, *pymnt_plan*, *purpose* and *term*. Finally *loan_status* seems to be more the output of our analysis, as we will better explain later.

In [24]:
# List of all the features names to be investigated
cols = ['emp_length', 'home_ownership', 'verification_status', 'pymnt_plan', 'purpose', 'term']

In [25]:
# collect in a dictionary, for each feature, the count of unique values 
values_counted = {}
count = []
for col in cols:
    values_counted[col] = loans_2007[col].value_counts()
    count.append(loans_2007[col].value_counts().count())

In [26]:
values_counted

{'emp_length': 10+ years    9357
 < 1 year     5005
 2 years      4727
 3 years      4354
 4 years      3635
 1 year       3568
 5 years      3452
 6 years      2370
 7 years      1869
 8 years      1587
 9 years      1337
 Name: emp_length, dtype: int64, 'home_ownership': RENT        19668
 MORTGAGE    18444
 OWN          3013
 OTHER         134
 NONE            2
 Name: home_ownership, dtype: int64, 'purpose': debt_consolidation    19327
 credit_card            5319
 other                  4209
 home_improvement       3089
 major_purchase         2231
 small_business         1941
 car                    1558
 wedding                 990
 medical                 724
 moving                  598
 house                   410
 educational             402
 vacation                365
 renewable_energy         98
 Name: purpose, dtype: int64, 'pymnt_plan': n    41260
 y        1
 Name: pymnt_plan, dtype: int64, 'term':  36 months    30541
  60 months    10720
 Name: term, dtype: int64, 've

As seen from the previous analysis, each column contains multiple discrete values. However before proceeding converting these columns into categorical variable, we should clean the *emp_length* column and treat it as a numerical one since the values have ordering.

In particular:

In [27]:
values_counted['emp_length']

10+ years    9357
< 1 year     5005
2 years      4727
3 years      4354
4 years      3635
1 year       3568
5 years      3452
6 years      2370
7 years      1869
8 years      1587
9 years      1337
Name: emp_length, dtype: int64

In [28]:
# we can simply map each value into numbers, keeping the same order
map_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}


We assume that people who may have been working more than 10 years have only really worked for 10 years. We also assume that people who've worked less than a year or if the information is not available that they've worked for 0. This is a general heuristic but it's not perfect.

In [29]:
# map the dictionary in the dataframe
loans_2007= loans_2007.replace(map_dict)

In [30]:
# Select columns that are still of object type
loans_2007.dtypes[loans_2007.dtypes =='object']

term                   object
home_ownership         object
verification_status    object
loan_status            object
pymnt_plan             object
purpose                object
dtype: object

Now we can encode the categorical variables described in this section (in particular: 'home_ownership', 'verification_status', 'pymnt_plan', 'purpose' and 'term'). To this end three main steps are needed:

1) We need to create a new DataFrame containing a new column for each dummy variable;

2) Add these new dummy columns to the original DataFrame;

3) Remove the original columns

In [31]:
cols = ['home_ownership', 'verification_status', 'pymnt_plan', 'purpose',  'term']
# create a new DataFrame
dummy_df = pd.get_dummies(loans_2007[cols])
# add new df to the original one
loans = pd.concat([loans_2007, dummy_df], axis=1)
# remove old columns
loans_2007 = loans_2007.drop(cols, axis=1)

In [32]:
df2table(loans_2007.head())

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,acc_now_delinq,tax_liens
0,5000,10.65,162.87,10,24000,Fully Paid,27.65,0,1,3,0,13648,83.7,9,0,0
1,2500,15.27,59.83,0,30000,Charged Off,1,0,5,3,0,1687,9.4,4,0,0
2,2400,15.96,84.33,10,12252,Fully Paid,8.72,0,2,2,0,2956,98.5,10,0,0
3,10000,13.49,339.31,10,49200,Fully Paid,20,0,1,10,0,5598,21,37,0,0
4,3000,12.69,67.79,1,80000,Current,17.94,0,0,15,0,27783,53.9,38,0,0


#### Target Column

We now need to decide on a target column that we want to use for modeling.

We should use the *loan_status* column, since it's the only column that directly describes if a loan was paid off on time, had delayed payments, or was defaulted on the borrower. Currently, this column contains text values and we need to convert it to a numerical one for training a model. Let's explore the different values in this column and come up with a strategy for converting the values in this column.

In [33]:
# let us consider how many unique values are there in the loan_status feature
loan_status = pd.DataFrame()
loan_status = loans_2007['loan_status'].value_counts().to_frame()
loan_status['Count'] = loan_status['loan_status']
loan_status['Loan Status'] = loan_status.index
loan_status.reset_index(level=0, inplace=True)
loan_status.drop(['loan_status', 'index'], axis = 1, inplace = True)

We can add also the description of each value of the *loan_status* feature that can be found in the [lendingclub website](https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-).

In [34]:
descriptions = ['Loan has been fully paid off.',
               'Loan for which there is no longer a reasonable expectation of further payments.',
               'While the loan was paid off, the loan application today would no longer meet the credit policy and wouldn t be approved on to the marketplace.',
               'While the loan was charged off, the loan application today would no longer meet the credit policy and wouldn t be approved on to the marketplace.',
               'The loan is past due but still in the grace period of 15 days.',
               'Loan hasn t been paid in 16 to 30 days (late on the current payment).',
               'Loan hasn t been paid in 31 to 120 days (late on the current payment).',
               'Loan is up to date on current payments.',
               'Loan is defaulted on and no payment has been made for more than 121 days.']
loan_status['Description'] = descriptions

In [35]:
df2table(loan_status)

,Count,Loan Status,Description
0,32258,Fully Paid,Loan has been fully paid off.
1,5390,Charged Off,Loan for which there is no longer a reasonable expectation of further payments.
2,1906,Does not meet the credit policy. Status:Fully Paid,"While the loan was paid off, the loan application today would no longer meet the credit policy and wouldn t be approved on to the marketplace."
3,923,Current,"While the loan was charged off, the loan application today would no longer meet the credit policy and wouldn t be approved on to the marketplace."
4,733,Does not meet the credit policy. Status:Charged Off,The loan is past due but still in the grace period of 15 days.
5,23,Late (31-120 days),Loan hasn t been paid in 16 to 30 days (late on the current payment).
6,18,In Grace Period,Loan hasn t been paid in 31 to 120 days (late on the current payment).
7,8,Late (16-30 days),Loan is up to date on current payments.
8,2,Default,Loan is defaulted on and no payment has been made for more than 121 days.


There are 8 different possible values for the loan_status column. Only the *Fully Paid* and *Charged Off* values describe the final outcome of the loan. Since from the investor's perspective, we're interested in trying to predict which loans will be paid off on time and which ones won't be, we need to convert the output into a binary vector that contains all the loans that have been paid (*Fully Paid*, can be reported as 1) and the loans that will not be paid (*Charged Off*, that will be associated to the value 0).

In [36]:
# Filter Dataframe considering only the outputs Fully Paid and Charged Off
loans_2007_filt = loans_2007[(loans_2007['loan_status']=='Fully Paid') | (loans_2007['loan_status']=='Charged Off')]

In [37]:
# Convert values in loan_status feature in binary numbers
map_dict = {
    
    "loan_status":{
        "Fully Paid" : 1,
        "Charged Off" : 0
    }
}
loans_2007_filt = loans_2007_filt.replace(map_dict)

In [38]:
df2table(loans_2007_filt.head())

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,acc_now_delinq,tax_liens
0,5000,10.65,162.87,10,24000,1,27.65,0,1,3,0,13648,83.7,9,0,0
1,2500,15.27,59.83,0,30000,0,1,0,5,3,0,1687,9.4,4,0,0
2,2400,15.96,84.33,10,12252,1,8.72,0,2,2,0,2956,98.5,10,0,0
3,10000,13.49,339.31,10,49200,1,20,0,1,10,0,5598,21,37,0,0
5,5000,7.9,156.46,3,36000,1,11.2,0,3,9,0,7963,28.3,12,0,0


Lastly, one thing we need to keep in mind is the class imbalance between the positive and negative cases. While there are 33,136 loans that have been fully paid off, there are only 5,634 that were charged off. This class imbalance is a common problem in binary classification and during training, the model ends up having a strong bias towards predicting the class with more observations in the training set and will rarely predict the class with less observations. The stronger the imbalance, the more biased the model becomes. So we need to keep the class imbalance in mind as we build machine learning models.

## Making Predictions

The cleaning process was necessary in order to generate features from the data, which can feed into a machine learning algorithm. The algorithm will make predictions about whether or not a loan will be paid off on time, which is contained in the loan_status column of the clean dataset.

In [39]:
# Let us have a look at the cleaned dataframe
loans_2007_filt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37648 entries, 0 to 39753
Data columns (total 16 columns):
loan_amnt         37648 non-null float64
int_rate          37648 non-null float64
installment       37648 non-null float64
emp_length        37648 non-null int64
annual_inc        37648 non-null float64
loan_status       37648 non-null int64
dti               37648 non-null float64
delinq_2yrs       37648 non-null float64
inq_last_6mths    37648 non-null float64
open_acc          37648 non-null float64
pub_rec           37648 non-null float64
revol_bal         37648 non-null float64
revol_util        37648 non-null float64
total_acc         37648 non-null float64
acc_now_delinq    37648 non-null float64
tax_liens         37648 non-null float64
dtypes: float64(14), int64(2)
memory usage: 4.9 MB


### Error Metric

When building a prediction model one should first make clear the question to address in order to evaluate which prediction problem (e.g. regression or classification) needs to be set. After that we can select an error metric.   

As stated at the beginning the question we would like to addred is the following:

**Will a borrower pay off his/her loan on time?**

This question can be easily associated to a binary classification (yes/no) problem. For this type of problem one can think of several error metric, e.g. accuracy, precision, recall, F-score etc.
In order to correctly select one of these it is important to deeply uderstand the problem to address and what we want to penalize during the selection of a machine learning algortihm. For example in the case of the precision we give more importance to the *false positive*, so in our case all the borrower that, according to the algortihm, will pay his/her loan but that in reality will not. Or, on the other hand, the recall emphasizes the *false negative* etc.

Our objective in this project is to make money -- we want to fund enough loans that are paid off on time to offset our losses from loans that aren't paid off. An error metric will help us determine if our algorithm will make us money or lose us money. This means that we should be concerned both on the false positive and false negative. This loses us potential money, since we didn't fund a loan that actually would have been paid off. However from an investor point of view, in order to minimize the risk, we may want to minimize the number of false positive. They'd be more okay with missing out on opportunities (false negatives) than they would be with funding a risky loan (false positives).
According to the previous reasoning it is clear that recall, as error metric, would be not the best one since it emphasizes the *false negative*; the accuracy also should be not the best choice expecially because we have unbalanced classes thus we could easily have an high accuracy although we are predicting as "good" borrower someone who will not pay back. The natural choice for the error metric is, then,the **precision**, defined as follows:
\begin{align}
P = \frac{True Positive}{True Positive + False Positive}
\end{align}


Although the precision is quite good measures since we want to reduce the selection of candidates who will not pay for their loan, it is possible that we are also reducing the number of good candidates that will be actually selected (borrowers who will pay for their loan). If one wants also to consider the contribution of borrower that the model suggest as 'delinquent' but who, in reality, will pay back the loan (false negative), one should use the F1-score, that considers both the precision (that we already use for the first part) and the recall, that is defined as follows:
\begin{align}
R = \frac{True Positive}{True Positive + False Negative}.
\end{align}

The F1-score is a weighted average of the precision and the recall.

### Machine Learning Algorithm

Now that we've setup error metrics, we can move on to making predictions using a machine learning algorithm. 
Before we can write a useful function for cross_validation that can be used for different types of models.

In [40]:
def cross_validate(model, features, labels, folds):
    # Let us use KFold to provide train/test indices to split data into train/test sets
    kf = KFold(n_splits=folds, shuffle=True, random_state=3)
    prec_values_test=[]
    prec_values_train=[]
    f1_values_test=[]
    f1_values_train=[]
    iterat = 0
    train_index_vec=[]
    predictions_dict = {}
    real_dict = {}
    # Iterate through over each fold.
    for train_index, test_index in kf.split(features):
        iterat = iterat+1
        print('iteration #' + str(iterat))
    # Training and test sets.
        X_train, X_test = features.iloc[train_index].reset_index(drop=True), features.iloc[test_index].reset_index(drop=True)
        y_train, y_test = labels.iloc[train_index].reset_index(drop=True), labels.iloc[test_index].reset_index(drop=True)
        #train_index_vec.append(train_index)
        
    # Fit the model and make predictions using both the test and train sets
        model.fit(X_train, y_train)
        predictions_test = model.predict(X_test)
        predictions_train = model.predict(X_train)
        predictions_dict[iterat] = predictions_test
        real_dict[iterat] = y_test.values
    # Calculate mse and accuracy values for this fold (train and test).
        prec_test = precision_score(y_test, predictions_test)
        prec_train = precision_score(y_train, predictions_train)
        f1_test = f1_score(y_test, predictions_test)
        f1_train = f1_score(y_train, predictions_train)

    # Append to arrays to calculate overall average precision and f1-score values.
        prec_values_test.append(prec_test)
        prec_values_train.append(prec_train)
        f1_values_test.append(f1_test)
        f1_values_train.append(f1_train)
    # compute and return average precision and f1-score squared error for the train and test sets.    
    avg_prec_test = np.mean(prec_values_test)
    avg_prec_train = np.mean(prec_values_train)
    avg_f1_test = np.mean(f1_values_test)
    avg_f1_train = np.mean(f1_values_train)
    return(avg_prec_test, avg_prec_train, avg_f1_test, avg_f1_train, predictions_dict, real_dict)
    

#### Logistic Regression

A good first algorithm to apply to binary classification problems is logistic regression, for several reasons: first of all it's quick to train and we can iterate more quickly; moreover it's less prone to overfitting than more complex models like decision trees and finally it's easy to interpret.

In [41]:
features = loans_2007_filt.drop('loan_status', axis = 1)
target = loans_2007_filt['loan_status']

In [42]:
# let use define the model we want to use
model = LogisticRegression(class_weight="balanced")
avg_prec_test, avg_prec_train, avg_f1_test, avg_f1_train ,predictions_dict, real_dict = cross_validate(model, features, target, 4)
print 'The precision in predicting the test set is: ',avg_prec_test, 'while the F1-score is: ', avg_f1_test

iteration #1


/Users/jessicalanini/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


iteration #2
iteration #3
iteration #4
The precision in predicting the test set is:  0.9132303209063808 while the F1-score is:  0.748886981075274


As we have discussed before the target vector has imbalanced classes; in particular:

In [43]:
target.value_counts()

1    32258
0     5390
Name: loan_status, dtype: int64

In general if we would like to get a classifier to correct for imbalanced classes. The two main ways are:

- Use oversampling and undersampling to ensure that the classifier gets input that has a balanced number of each class.
- Tell the classifier to penalize misclassifications of the less prevalent class more than the other class.

The simplest way to proceed is with the second strategy. This is possible by simply setting the class_weight parameter to balanced when creating the LogisticRegression instance.
We can now repeate the previous evaluation.

In [44]:
# let use define the model we want to use
model = LogisticRegression(class_weight="balanced")
avg_prec_test, avg_prec_train, avg_f1_test, avg_f1_train ,predictions_dict, real_dict = cross_validate(model, features, target, 4)
print 'The precision in predicting the test set is: ',avg_prec_test, 'while the F1-score is: ', avg_f1_test

iteration #1
iteration #2
iteration #3
iteration #4
The precision in predicting the test set is:  0.9126530064140187 while the F1-score is:  0.75497967202708


As we can see by using the option "balanced" the performances did not improve, which means that our algortihm was already good in handling the unbalanced classes.

In general the precision of the model is quite high (>90%) which means that we are reducing the false positive that the model predicts. From a conservative investor's standpoint, it's reassuring that the precision is high because it means that we'll be able to do a better job at avoiding bad loans than if we funded everything. Moreover the F1-score, although is not that low, it is lower than the precision, which means that we are losing few potential good borrowers.

We can try to increase the precision further by assigning a harsher penalty for misclassifying the negative class. While setting class_weight to balanced will automatically set a penalty based on the number of 1s and 0s in the column, we can also set a manual penalty, as follows:

In [45]:
penalty = {
    0: 10,
    1: 1
}
model = LogisticRegression(class_weight=penalty)

In [46]:
avg_prec_test, avg_prec_train, avg_f1_test, avg_f1_train ,predictions_dict, real_dict = cross_validate(model, features, target, 4)
print 'The precision in predicting the test set is: ',avg_prec_test, 'while the F1-score is: ', avg_f1_test

iteration #1
iteration #2
iteration #3
iteration #4
The precision in predicting the test set is:  0.9460275166994156 while the F1-score is:  0.3498663999108288


Although, thanks to the hasher penality the precision is quite good, and thus we lowered our risk, the F1-score has reduced a lot, showing that we are most probably increasing the *false negative*. In other words we're also missing opportunities to fund more loans and potentially make more money.

## Conclusion

In this project we would like to build a predictive model able to tell whether a borrower of a loan will pay it back or not. This is important for investors who will eventually provide the money and thus they would like to minimize the risk to have loans that are not paid off on time. 

As first steps we have imported a data set from a marketplace for personal loans that matches borrowers who are seeking a loan with investors looking to lend money and make a return, called Lending Club. We cleaned and analyzed this dataset by handling missing values, remove not useful features and covert important features. 
After that this problem has been modeled as a binary classification problem and a logistic regression was used as model to predict possible 'bad' borrower. Several variants of such a logistic regression model has been tested and the one with the highest precision and accuracy has been selected in order to still consider as most important aspect the minimization of *false positive* (e.g. borrowers that the model select as the ones who will paid their loan on time but who in reality will be delinquent) but without losing a lot of good borrower (*false negative*). 